If in VSCODE:

In [1]:
# check if Torch supports CUDA
import torch
print(torch.__version__)
torch.cuda.is_available()


2.1.0+cu121


True

Hyper params

In [2]:
# The model that you want to train from the Hugging Face hub
model_name = "codellama/CodeLlama-7b-Instruct-hf"

# The instruction dataset to use
dataset_name = "StarkWizard/cairo-instruct"

# Fine-tuned model name
new_model = "StarkWizard/llama-2-7b-cairo-trained-PEFT"


import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"



nb_epochs = 6

In [3]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="codellama-cairo",
    
    # track hyperparameters and run metadata
    config={

    "epochs":nb_epochs,
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pierre-emmanuel-chaut. Use `wandb login --relogin` to force relogin


Load Base Model and Tokenizer

In [4]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM,prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer
import os

# Load the tokenizer from the model (llama2)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token =tokenizer.eos_token
tokenizer.padding_side='right'

# load the quantized settings: 4 bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)


# don't use the cache
model.config.use_cache = False
model.config.pretraining_tp=1
model.config.window = 256 
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
        r=8, lora_alpha=32, lora_dropout=0.1, bias="none", task_type="CAUSAL_LM", target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "up_proj",
        "down_proj",
        "gate_proj"],
        inference_mode = False
    )



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Import Dataset from Hugging Face

In [6]:
from datasets import load_dataset

# Load the dataset
dataset_train = load_dataset(dataset_name, split="train", download_mode='force_redownload',ignore_verifications=True)
dataset_test = load_dataset(dataset_name, split="eval", download_mode='force_redownload',ignore_verifications=True)
dataset_train = dataset_train.shuffle(seed=42)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3196 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/5 [00:00<?, ? examples/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3196 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/5 [00:00<?, ? examples/s]

Check to see if everything is fine before launching training

Load Model

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
from trl import SFTTrainer
import transformers



supervised_finetuning_trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=1024,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
         gradient_checkpointing = True,
         evaluation_strategy="steps",
        warmup_steps=5,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=50,
        num_train_epochs=nb_epochs,
        output_dir=new_model,
        fp16=False,
        push_to_hub=True,
        group_by_length=True,
         report_to="wandb",
    ),
)


Map:   0%|          | 0/3196 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [8]:
torch.cuda.empty_cache()
supervised_finetuning_trainer.train()
supervised_finetuning_trainer.model.push_to_hub(new_model)
tokenizer.push_to_hub(new_model)

  0%|          | 0/4794 [00:00<?, ?it/s]

You're using a CodeLlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/pechaut/miniconda3/envs/cairo-llm/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.7928, 'learning_rate': 0.00019995643140403337, 'epoch': 0.06}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 1.9221851825714111, 'eval_runtime': 1.0441, 'eval_samples_per_second': 4.789, 'eval_steps_per_second': 0.958, 'epoch': 0.06}
{'loss': 1.4181, 'learning_rate': 0.00019980587265177913, 'epoch': 0.13}


...

{'eval_loss': 1.3831690549850464, 'eval_runtime': 1.0435, 'eval_samples_per_second': 4.792, 'eval_steps_per_second': 0.958, 'epoch': 5.13}
{'loss': 0.2125, 'learning_rate': 8.79190293666845e-06, 'epoch': 5.19}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 1.3588929176330566, 'eval_runtime': 1.073, 'eval_samples_per_second': 4.66, 'eval_steps_per_second': 0.932, 'epoch': 5.19}
{'loss': 0.2134, 'learning_rate': 7.496365678581874e-06, 'epoch': 5.26}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 1.3508223295211792, 'eval_runtime': 1.0614, 'eval_samples_per_second': 4.711, 'eval_steps_per_second': 0.942, 'epoch': 5.26}


If you are on a colab, push to hub or save to drive